# Feature engineering

> Jean Cardoso, Ph.D.

## Carregando dados

In [ ]:
# Carregando o notebook externo
!pip install -q import-ipynb

In [ ]:
import import_ipynb

#from google.colab import drive
#drive.mount('/content/drive')

# Caminho para o notebook `features.ipynb` no Google Drive
notebook_path = '/content/drive/MyDrive/case_rentcars/case_rentcars/features.ipynb'

In [ ]:
# Importando as funções do notebook
import nbformat
from nbconvert import PythonExporter

with open(notebook_path) as f:
    nb = nbformat.read(f, as_version=4)

exporter = PythonExporter()
source, _ = exporter.from_notebook_node(nb)
exec(source)

In [ ]:
#Carregando bibliotecas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

# Configurações do pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Carregando o conjunto de dados
aisles = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/departments.csv')
products = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/products.csv')
orders = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/orders.csv')

prior = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/order_products__prior.csv')
train = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/order_products__train.csv')


In [ ]:
# Unir os dados de produtos com aisles e departments
products = products.merge(aisles, on='aisle_id').merge(departments, on='department_id')

# Unir os dados de orders com prior
prior_orders = orders.merge(prior, on='order_id')
prior_orders = prior_orders.merge(products, on='product_id')

In [ ]:
# Verificando valores NaN
prior_orders.isnull().sum()

order_id                        0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
department                      0
dtype: int64

In [ ]:
# days_since_prior_order: Número de dias desde o pedido anterior
# (NaN para o primeiro pedido).
prior_orders['days_since_prior_order'].value_counts(dropna=False, normalize=True)

days_since_prior_order
7.0     0.107278
30.0    0.094654
6.0     0.077693
5.0     0.065560
4.0     0.064147
NaN     0.064070
8.0     0.059622
3.0     0.057898
2.0     0.045164
9.0     0.037583
14.0    0.031775
10.0    0.029849
1.0     0.029016
13.0    0.026265
11.0    0.025328
12.0    0.023890
15.0    0.020721
16.0    0.014238
0.0     0.013834
21.0    0.013706
17.0    0.011792
20.0    0.011718
18.0    0.010819
19.0    0.010334
22.0    0.009531
28.0    0.007811
23.0    0.006969
27.0    0.006297
24.0    0.005969
25.0    0.005568
26.0    0.005486
29.0    0.005415
Name: proportion, dtype: float64

In [ ]:
# Criar a coluna indicadora para o primeiro pedido
prior_orders['is_first_order'] = prior_orders['days_since_prior_order'].isna().astype(int)

# Substituir NaN por zero (ou -1, se preferir)
prior_orders['days_since_prior_order'].fillna(0, inplace=True)

In [ ]:
# Conferindo valores faltandos
prior_orders.isnull().sum()

order_id                  0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
product_name              0
aisle_id                  0
department_id             0
aisle                     0
department                0
is_first_order            0
dtype: int64

In [ ]:
# Organizando as colunas
col_order = ['user_id',
            'order_id',
            'product_id',
            'aisle_id',
            'department_id',
            'add_to_cart_order',
            'reordered',
            'product_name',
            'aisle',
            'department',
            'eval_set',
            'order_number',
            'order_dow',
            'order_hour_of_day',
            'days_since_prior_order',
            'is_first_order']

prior_orders = prior_orders[col_order]
print(prior_orders.shape)
prior_orders.head()

(32434489, 16)


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,is_first_order
0,1,2539329,196,77,7,1,0,Soda,soft drinks,beverages,prior,1,2,8,0.0,1
1,1,2398795,196,77,7,1,1,Soda,soft drinks,beverages,prior,2,3,7,15.0,0
2,1,473747,196,77,7,1,1,Soda,soft drinks,beverages,prior,3,3,12,21.0,0
3,1,2254736,196,77,7,1,1,Soda,soft drinks,beverages,prior,4,4,7,29.0,0
4,1,431534,196,77,7,1,1,Soda,soft drinks,beverages,prior,5,4,15,28.0,0


In [ ]:
# Analisando o consumo de memória
prior_orders.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 int64  
 1   order_id                int64  
 2   product_id              int64  
 3   aisle_id                int64  
 4   department_id           int64  
 5   add_to_cart_order       int64  
 6   reordered               int64  
 7   product_name            object 
 8   aisle                   object 
 9   department              object 
 10  eval_set                object 
 11  order_number            int64  
 12  order_dow               int64  
 13  order_hour_of_day       int64  
 14  days_since_prior_order  float64
 15  is_first_order          int64  
dtypes: float64(1), int64(11), object(4)
memory usage: 11.4 GB


In [ ]:
# Aplicar otimização de memória
prior_orders = reduce_memory_usage(prior_orders)

In [ ]:
# Salvando os dados no formato reduzido e em parquet
prior_orders.to_parquet('/content/drive/MyDrive/case_rentcars/data/interim/prior_orders.parquet', compression='snappy')

In [ ]:
# Carrengando dados reduzidos
prior_orders =  pd.read_parquet('/content/drive/MyDrive/kaggle/feature_enginier/prior_orders.parquet')
print(prior_orders.shape)
prior_orders.head()

(32434489, 16)


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,is_first_order
0,1,2539329,196,77,7,1,0,Soda,soft drinks,beverages,prior,1,2,8,0.0,1
1,1,2398795,196,77,7,1,1,Soda,soft drinks,beverages,prior,2,3,7,15.0,0
2,1,473747,196,77,7,1,1,Soda,soft drinks,beverages,prior,3,3,12,21.0,0
3,1,2254736,196,77,7,1,1,Soda,soft drinks,beverages,prior,4,4,7,29.0,0
4,1,431534,196,77,7,1,1,Soda,soft drinks,beverages,prior,5,4,15,28.0,0


In [ ]:
# Analisando a memória
prior_orders.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 16 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   user_id                 int32   
 1   order_id                int32   
 2   product_id              int32   
 3   aisle_id                int16   
 4   department_id           int8    
 5   add_to_cart_order       int16   
 6   reordered               int8    
 7   product_name            category
 8   aisle                   category
 9   department              category
 10  eval_set                category
 11  order_number            int8    
 12  order_dow               int8    
 13  order_hour_of_day       int8    
 14  days_since_prior_order  float32 
 15  is_first_order          int8    
dtypes: category(4), float32(1), int16(2), int32(3), int8(6)
memory usage: 1.0 GB


## Gerando novas features

In [ ]:
%%time
# Gerar features
user_features = generate_user_features(prior_orders)
purchase_features = generate_purchase_features(prior_orders)
temporal_features = generate_temporal_features(prior_orders)
user_session_features, product_days_diff = generate_session_features(prior_orders)
behavior_features = generate_behavior_features(prior_orders)
product_features = generate_product_features(prior_orders)
product_features_2 = generate_product_features_2(prior_orders)

CPU times: user 1min 24s, sys: 8.69 s, total: 1min 33s
Wall time: 1min 32s


In [ ]:
%%time
# Aplicar otimização de memória
behavior_features = reduce_memory_usage(behavior_features)
user_features = reduce_memory_usage(user_features)
purchase_features = reduce_memory_usage(purchase_features)
temporal_features = reduce_memory_usage(temporal_features)
user_session_features = reduce_memory_usage(user_session_features)
product_days_diff = reduce_memory_usage(product_days_diff)
product_features = reduce_memory_usage(product_features)


CPU times: user 110 ms, sys: 18.9 ms, total: 129 ms
Wall time: 128 ms


## Gerando dados de treino

In [ ]:
#train = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/order_products__train.csv')
#orders = pd.read_csv('/content/drive/MyDrive/case_rentcars/data/interim/orders.csv')
orders = reduce_memory_usage(orders)
train = reduce_memory_usage(train)

In [ ]:
%%time
# Filtrar apenas os dados de treino
train_orders = orders[orders['eval_set'] == 'train'].merge(train, on='order_id')

# Associar features de usuário aos dados de treino
train_orders = train_orders.merge(user_features, on='user_id', how='left')
train_orders = train_orders.merge(behavior_features, on='user_id', how='left')
train_orders = train_orders.merge(temporal_features, on='user_id', how='left')
train_orders = train_orders.merge(user_session_features, on='user_id', how='left')

# Associar features de produto e sessão aos dados de treino
train_orders = train_orders.merge(product_features.groupby('product_id').mean().drop('user_id', axis=1), on='product_id', how='left')
train_orders = train_orders.merge(product_days_diff, on='product_id', how='left')
train_orders = train_orders.merge(purchase_features, on='product_id', how='left')


CPU times: user 1.25 s, sys: 90.7 ms, total: 1.34 s
Wall time: 1.34 s


In [ ]:
# Verificando valores NaN
train_orders.isna().sum()

order_id                         0
user_id                          0
eval_set                         0
order_number                     0
order_dow                        0
order_hour_of_day                0
days_since_prior_order           0
product_id                       0
add_to_cart_order                0
reordered                        0
user_order_count                 0
user_avg_products_per_order      0
user_unique_products             0
user_reorder_ratio               0
user_avg_days_between_orders     0
user_reorder_rate                0
user_reorder_freq                0
user_reordered_products_ratio    0
order_dow_sin                    0
order_dow_cos                    0
order_hour_of_day_sin            0
order_hour_of_day_cos            0
user_hour_order_freq             0
user_dow_order_freq              0
user_avg_days_since_prior        0
user_days_since_last_order       0
product_popularity               9
user_product_popularity          9
product_avg_days_sin

In [ ]:
# Preenchendo valores faltantes
train_orders['product_order_freq'] = train_orders['product_order_freq'].fillna(0)
train_orders['product_reorder_ratio'] = train_orders['product_reorder_ratio'].fillna(0)
train_orders['avg_pos_incart'] = train_orders['avg_pos_incart'].fillna(-1)
train_orders['product_avg_days_since_prior'] = train_orders['product_avg_days_since_prior'].fillna(train_orders['product_avg_days_since_prior'].mean())
train_orders['product_popularity'] = train_orders['product_popularity'].fillna(0)
train_orders['user_product_popularity'] = train_orders['user_product_popularity'].fillna(0)

In [ ]:
# Conferindo valores NaN
train_orders.isna().sum()

order_id                         0
user_id                          0
eval_set                         0
order_number                     0
order_dow                        0
order_hour_of_day                0
days_since_prior_order           0
product_id                       0
add_to_cart_order                0
reordered                        0
user_order_count                 0
user_avg_products_per_order      0
user_unique_products             0
user_reorder_ratio               0
user_avg_days_between_orders     0
user_reorder_rate                0
user_reorder_freq                0
user_reordered_products_ratio    0
order_dow_sin                    0
order_dow_cos                    0
order_hour_of_day_sin            0
order_hour_of_day_cos            0
user_hour_order_freq             0
user_dow_order_freq              0
user_avg_days_since_prior        0
user_days_since_last_order       0
product_popularity               0
user_product_popularity          0
product_avg_days_sin

In [ ]:
train_orders.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 32 columns):
 #   Column                         Non-Null Count    Dtype   
---  ------                         --------------    -----   
 0   order_id                       1384617 non-null  int32   
 1   user_id                        1384617 non-null  int32   
 2   eval_set                       1384617 non-null  category
 3   order_number                   1384617 non-null  int8    
 4   order_dow                      1384617 non-null  int8    
 5   order_hour_of_day              1384617 non-null  int8    
 6   days_since_prior_order         1384617 non-null  float32 
 7   product_id                     1384617 non-null  int32   
 8   add_to_cart_order              1384617 non-null  int8    
 9   reordered                      1384617 non-null  int8    
 10  user_order_count               1384617 non-null  int8    
 11  user_avg_products_per_order    1384617 non-null  float32 
 12  

In [ ]:
# Salvando os dados no formato reduzido e em parquet
train_orders.to_parquet('/content/drive/MyDrive/case_rentcars/data/processed/data_train.parquet', compression='snappy')

## Gerando dados de teste

In [64]:
%%time
# Filtrar apenas os dados de teste
test_orders = orders[orders['eval_set'] == 'test']

# Unir as features com os dados de teste
test_orders = test_orders.merge(behavior_features, on='user_id', how='left')
test_orders = test_orders.merge(user_features, on='user_id', how='left')
test_orders = test_orders.merge(user_session_features, on='user_id', how='left')
test_orders = test_orders.merge(temporal_features, on='user_id', how='left')
test_orders = test_orders.merge(product_features, on='user_id', how='left')
test_orders = test_orders.merge(product_days_diff, on='product_id', how='left')
test_orders = test_orders.merge(purchase_features, on='product_id', how='left')

CPU times: user 2.13 s, sys: 93.8 ms, total: 2.22 s
Wall time: 2.22 s


In [66]:
test_orders.isna().sum()

order_id                         0
user_id                          0
eval_set                         0
order_number                     0
order_dow                        0
order_hour_of_day                0
days_since_prior_order           0
user_reorder_rate                0
user_reorder_freq                0
user_reordered_products_ratio    0
user_order_count                 0
user_avg_products_per_order      0
user_unique_products             0
user_reorder_ratio               0
user_avg_days_between_orders     0
user_avg_days_since_prior        0
user_days_since_last_order       0
order_dow_sin                    0
order_dow_cos                    0
order_hour_of_day_sin            0
order_hour_of_day_cos            0
user_hour_order_freq             0
user_dow_order_freq              0
product_id                       0
product_popularity               0
user_product_popularity          0
product_avg_days_since_prior     0
product_order_freq               0
product_reorder_rati

In [71]:
# Salvando os dados no formato reduzido e em parquet
test_orders.to_parquet('/content/drive/MyDrive/case_rentcars/data/processed/data_test.parquet', compression='snappy')